# Семинар 6: Character-Level LSTM

## Вступление
На прошлом занятии мы познакомились с тем, как можно векторизовать текстовые данные для решения задач обработки текстов. Сегодня мы продолжим заниматься текстами и посмотрим на простейший пример автоматической генерации текстов при помощи Recurrent Neural Network (RNN).

Полезные материалы по RNN можно почитать [здесь](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), а реализацию на PyTorch — [здесь](https://github.com/karpathy/char-rnn).

### План семинара
1. Подготовка данных
2. Имплементация модели
3. Обучение модели
4. Применение модели

In [ ]:
!pip install lightning -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
from typing import Iterable, Tuple

import pytorch_lightning as pl
import torch
from torch import nn

warnings.filterwarnings("ignore")

## 1. Подготовка данных

### Загрузим текст "Анны Карениной"

In [ ]:
!wget https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt

--2025-05-17 15:12:05--  https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1985223 (1.9M) [text/plain]
Saving to: ‘anna.txt’

anna.txt            100%[===================>]   1.89M  --.-KB/s    in 0.04s   

2025-05-17 15:12:05 (43.8 MB/s) - ‘anna.txt’ saved [1985223/1985223]



In [ ]:
with open("anna.txt") as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Токенизируем текст

Аналогично предыдущему семинару, в ячейках ниже создадим два словаря для преобразования символов в целые числа и обратно.

In [ ]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = torch.tensor([char2int[ch] for ch in text])
encoded[:100]

tensor([23, 49, 32, 43, 48, 74,  0, 77, 12, 33, 33, 33, 36, 32, 43, 43, 54, 77,
        46, 32, 41, 55, 61, 55, 74, 22, 77, 32,  0, 74, 77, 32, 61, 61, 77, 32,
        61, 55, 26, 74, 19, 77, 74, 20, 74,  0, 54, 77, 62,  8, 49, 32, 43, 43,
        54, 77, 46, 32, 41, 55, 61, 54, 77, 55, 22, 77, 62,  8, 49, 32, 43, 43,
        54, 77, 55,  8, 77, 55, 48, 22, 77, 24, 70,  8, 33, 70, 32, 54,  5, 33,
        33,  3, 20, 74,  0, 54, 48, 49, 55,  8])

Посмотрим на схему char-RNN:
<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="30%">

Сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный маппинг), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте напишем функцию для этого преобразования.

#### Задание: допишите функцию one-hot кодирования последовательности

In [ ]:
def one_hot_encode(int_words: torch.Tensor, n_labels: int) -> torch.Tensor:
    """
    Creates one-hot representation matrix for a given batch of integer sequences
    :param int_words: tensor of ints, which represents current sequence; shape: [batch_size, seq_len]
    :param n_labels: vocabulary size (number of unique tokens in data)
    :return: one-hot representation of the input tensor; shape: [batch_size, seq_len, n_labels]
    """
    words_one_hot = (
        torch.nn.functional.one_hot(int_words, num_classes=n_labels)
        .to(torch.float32)
        .to(int_words.device)
    )
    return words_one_hot

In [ ]:
# testing the function
test_seq = torch.tensor([[3, 5, 1], [0, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

tensor([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


### Сформируем батчи
На простом примере батчи будут выглядеть так: мы возьмем закодированные символы и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>

**1. Отбросим часть текста, чтобы у нас были только полные батчи**

Каждый батч содержит $N \times M$ символов, где $N$ — это количество последовательностей в батче (`batch_size`), а $M$ — длина каждой последовательности (`seq_length`). Затем, чтобы получить общее количество батчей $K$, которое мы можем сделать из последовательности, нужно разделить длину последовательности на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из последовательности: $N \times M \times K$.

**2. Разделим текст на $N$ частей**

Этот шаг нужен, чтобы мы могли проходить по тексту окном размера `[batch_size, seq_len]`. Его можно реализовать при помощи простого `reshape`.

**3. Теперь, когда у нас готова матрица текста, мы можем двигаться по ней окном, чтобы получить батчи**

Из каждой позиции окна сформируем обучающие пары `(x, y)` следующим образом: $x$ — это все элементы окна кроме последнего столбца, а $y$ — это все элементы окна кроме первого столбца. Тем самым для каждого токена исходного текста мы будем предсказывать следующий за ним токен.

#### Задание: допишите функцию генерации батчей

In [ ]:
def get_batches(
    int_words: torch.Tensor, batch_size: int, seq_length: int
) -> Iterable[torch.Tensor]:
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    symbols_per_batch = batch_size * window_size
    batch_count = len(int_words) // symbols_per_batch
    int_words_to_use = int_words[: batch_count * symbols_per_batch]

    # 2. Reshape into batch_size rows
    int_words_to_use = int_words_to_use.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words_to_use.shape[1], window_size):
        x = int_words_to_use[:, position : position + seq_length]
        # So that for Chapter for C the target is h
        y = int_words_to_use[:, position + 1 : position + seq_length + 1]
        yield x, y

In [ ]:
# testing the function
test_batches = get_batches(encoded, 8, 50)
test_x, test_y = next(test_batches)
assert test_x.shape == test_y.shape
print(f"x:\n{test_x[:10, :10]}\n")
print(f"y:\n{test_y[:10, :10]}")

x:
tensor([[23, 49, 32, 43, 48, 74,  0, 77, 12, 33],
        [43, 43, 74, 76, 77, 32,  8, 76, 77, 22],
        [48, 49, 32, 48, 77, 55,  8, 22, 24, 61],
        [74, 70, 77, 48, 49, 24, 62,  2, 49, 48],
        [74,  0, 63, 77, 49, 74, 77, 49, 32, 76],
        [ 0, 63, 77, 70, 49, 24, 77, 70, 32, 22],
        [22, 48, 77, 53, 74, 77, 30, 24, 20, 74],
        [63, 77, 52, 61, 74, 21, 74, 54, 77, 52]])

y:
tensor([[49, 32, 43, 48, 74,  0, 77, 12, 33, 33],
        [43, 74, 76, 77, 32,  8, 76, 77, 22, 49],
        [49, 32, 48, 77, 55,  8, 22, 24, 61, 62],
        [70, 77, 48, 49, 24, 62,  2, 49, 48, 46],
        [ 0, 63, 77, 49, 74, 77, 49, 32, 76, 77],
        [63, 77, 70, 49, 24, 77, 70, 32, 22, 77],
        [48, 77, 53, 74, 77, 30, 24, 20, 74,  0],
        [77, 52, 61, 74, 21, 74, 54, 77, 52, 61]])


### Наконец, подготовим класс датасета

In [ ]:
class AnnaData(torch.utils.data.IterableDataset):
    def __init__(self, int_words: torch.Tensor, batch_size: int, seq_length: int):
        self.int_words = int_words
        self.batch_size = batch_size
        self.seq_length = seq_length

    def __iter__(self):
        return get_batches(self.int_words, self.batch_size, self.seq_length)

## 2. Имплементация модели

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=50%>

### Структура модели

* Создаём и храним необходимые словари.
* Определяем слой [LSTM]((https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)) с помощью инстанса класса `torch.nn.LSTM`, который принимает набор параметров: `input_size` — длина последовательности в батче; `n_hidden` — размер скрытых слоёв; `n_layers` — количество слоёв; `drop_prob` — вероятность дропаута; и `batch_first` — флаг, указывающий на то, что у входных последовательностей размерность батча идёт вдоль нулевой оси.
* Определяем слой Dropout с таким же значением `drop_prob`.
* Определяем полносвязный слой с набором параметров: размерность ввода — `n_hidden`; размерность выхода — размер словаря.
* Наконец, инициализируем веса и начальное скрытое состояние (`self.init_hidden()`).

In [ ]:
class CharRNN(nn.Module):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 256,
        n_layers: int = 2,
        drop_prob: float = 0.5,
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        # create mappings
        self.unique_tokens = unique_tokens
        self.int2char = dict(enumerate(self.unique_tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## define the LSTM, dropout and fully connected layers
        self.lstm = nn.LSTM(
            input_size=len(unique_tokens),
            hidden_size=n_hidden,
            num_layers=n_layers,
            dropout=drop_prob,
            batch_first=True,
        )
        self.fc = nn.Linear(n_hidden, len(unique_tokens))
        self.drop = nn.Dropout(drop_prob)

    def forward(
        self, x: torch.Tensor, hidden: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        out, hidden = self.lstm(x, hidden)
        # Stack up LSTM outputs using view. You may need to use contiguous to reshape the output.
        out = self.drop(out)
        out = out.contiguous().view(-1, self.n_hidden)
        ## Get the output for classification.
        out = self.fc(out)

        return out, hidden

    def init_hidden(
        self, batch_size: int, weight_device: torch.device
    ) -> Tuple[torch.Tensor]:
        """
        Creates two new zero tensors for hidden state and cell state of LSTM
        :param batch_size: number of sequences per batch
        :param weight_device: torch.device("cuda") for GPU init or torch.device("cpu") for CPU init
        :return: tuple of two tensors of shape [n_layers x batch_size x n_hidden]
        """
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
        )
        return hidden

In [ ]:
class CharRNNModule(pl.LightningModule):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 1024,
        n_layers: int = 2,
        drop_prob: float = 0.5,
        batch_size: int = 128,
        seq_length=256,
        lr: float = 0.001,
    ) -> None:
        super().__init__()
        self.model = CharRNN(unique_tokens, n_hidden, n_layers, drop_prob)
        self.hidden = None
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.n_chars = len(unique_tokens)
        self.batch_size = batch_size
        self.seq_length = seq_length

    def training_step(
        self, train_batch: Tuple[torch.Tensor, torch.Tensor]
    ) -> torch.Tensor:
        x, y = train_batch
        x, y = x.squeeze(0), y.squeeze(0)
        x = one_hot_encode(x, self.n_chars)

        if self.hidden is None:
            self.hidden = self.model.init_hidden(self.batch_size, self.device)
        self.hidden = tuple([each.data for each in self.hidden])

        output, self.hidden = self.model(x, self.hidden)
        loss = self.loss(output, y.reshape(self.batch_size * self.seq_length).long())

        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

## 3. Обучение модели

По классике, используем оптимизатор Adam и кросс-энтропию. Но без пары особенностей не обойтись:
* Во время цикла будем отделять скрытое состояние от его истории, потому что скрытое состояние LSTM является кортежем скрытых состояний.
* Будем использовать gradient clipping, чтобы избавиться от взрывающихся градиентов.

In [ ]:
# data
train_dataset = AnnaData(encoded, batch_size=128, seq_length=256)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,  # batching is already implemented on our side
    shuffle=False,
    num_workers=1,  # don't change: it will lead to the wrong implementation
)
# model
char_rnn = CharRNNModule(unique_chars, n_hidden=1024, batch_size=128)
# trainer
trainer = pl.Trainer(max_epochs=15, gradient_clip_val=1.0, accelerator="gpu", devices=1)
trainer.fit(char_rnn, train_dataloaders=train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | CharRNN          | 13.0 M | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
52.097    Total estimated model params size (MB)
5         Modules in train mode
0         Modules i

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


## 4. Применение модели

Сперва сохраним обученную модель, чтобы можно было загрузить её позже. В следующей ячейке сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [ ]:
net = char_rnn.model
checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.unique_tokens,
}

with open("rnn_x_epoch.net", "wb") as f:
    torch.save(checkpoint, f)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Делаем предсказания

Сгенерируем текст! Для предсказания продолжения текста мы передаём в сеть последний символ, она предсказывает следующий символ, который мы снова передаем на вход, получаем ещё один предсказанный символ и так далее. Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов на каждом шаге генерации, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые, наиболее вероятные символы. С одной стороны, такой подход позволит нам рассматривать не только самую вероятную последовательность с точки зрения прогноза модели. С другой стороны, мы будем работать с ограниченным набором сгенерированных вариантов, поэтому избавимся от совсем уж шумовых прогнозов. Узнать больше можно [здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [ ]:
def predict_next_char(
    model: torch.nn.Module, char: str, h: torch.Tensor = None, top_k: int = None
) -> Tuple[str, torch.Tensor]:
    """
    Given a character and a model, predicts next character in the sequence
    :param model: model that outputs next token probability distribution
    :param char: last character of the sequence to continue generation from
    :param h: hidden state of the model
    :param top_k: number of most probable tokens to be chosen from
    :return: tuple of next character and new hidden state
    """
    # tensor inputs
    x = torch.tensor([[model.char2int[char]]])
    x = one_hot_encode(x, len(model.unique_tokens))
    x = x.to(device)

    h = tuple([each.data for each in h])

    out, h = model(x, h)

    # get the character probabilities
    p = torch.nn.functional.softmax(out, dim=1).data.cpu()

    # get top characters
    if top_k is None:
        top_ch = torch.arange(len(model.unique_tokens))
    else:
        p, top_ch = p.topk(top_k)

    p.squeeze_()
    top_ch.squeeze_()
    char = top_ch[torch.multinomial(p / p.sum(), 1)]

    return model.int2char[char.item()], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [ ]:
def sample(model, size, prime="The", top_k=None):
    model.to(device)
    model.eval()

    # run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1, device)
    for ch in prime:
        char, h = predict_next_char(model, ch, h, top_k=top_k)

    chars.append(char)

    # pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict_next_char(model, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [ ]:
print(sample(net, 1000, prime="Chapter", top_k=5))

Chapter with the particularly people, the
starratomen she his brother.

"I hope I
don't went to make the mother the coming over the same wouldnets," she said.

"This may say that I can't all when.... And it we thought of my friend
with a
carriage of it?"

"No otely to see the stop in a little of anything in the clotest--it
wish betway to be all to be set and always to delighted that she would come by it. And how counting him and he had
been doing it on if the cape, and would
never be to be meaning to see it in the morning, in
a shame. Hos sister's soon, then almott ago," he wanted to himself.

"I have to districuse my son," said Stepan Arkadyevitch.

"It means, I don't see her what we
say, I don't want to the strett on it,"-said Anna, "and if you want to
say at the propost, but in the carriage well, and talk a money together."

"What she was the people are, in sports is in the same, but I see, it must
say that?"

"I should be through about
the compatery?" she repeated,
"and he can't sa

### Загрузка чекпоинта и генерация

In [ ]:
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

# sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said at off that he had been sens the crush of which with the more of their hundred too.

The marshal, who had been told
him along a footman, with her head that his shoulder at the conversation what the people, all he could to
be begon to be in thinking at her father's
convinced that she went out of them," said Levin, looding the look and whingered in his while--at the same hand that
she was that it was in the children in that
some stard what with
such his she had taken too, but he was not the same at
the time he was so much to say what he saw that that this mare there was one
of the marshal of the whole triel. The way to her.

"All the maid as there.".. They woman are said when he was confisinced and a fore of her heart to be as a position, which he had taken her side he felt a wife he could not come to see hir sevenal
sort. But the corner the short out of them and said his father ten this work.,
and was in chaig at to see his fainher, but that the minute had
never but imagi